# Notebook activity A1.7 Estimating pooled OLS

In this notebook you will now make full use of the panel dataset `psid` to estimate a Mincerian equation. This is the first of three notebooks where you will be practicing the use of estimators that are available for panel data. In this notebook you will learn how to apply the pooled OLS estimator.

Recall that the data in `psid` corresponds to a balanced panel on 595 individuals over 7 time periods. As a reminder, the variables are described as follows.

* `exper`: the number of years of full-time experience
* `occupation`: taking values 1 (blue-collar occupation) and 0 (other occupations)
* `gender`: taking values 0 (male) and 1 (female)
* `educ`: the number of years of education
* `ethnicity`: taking values 1 (black) and 0 (otherwise)
* `wageLog`: natural logarithm of hourly wages (in cents of a dollar)
* `period`: year of the study that the observation relates to
* `id`: individual.


The Mincerian equation we will concentrate on is, for the $i$ individual in time period $t$,

$\texttt{wageLog}_{it} = \beta_0 + \beta_1 \texttt{educ}_{i} + \beta_2 \texttt{exper}_{it} +  \beta_3\texttt{exper}^2_{it} +\beta_4\texttt{occupation}_{it} + \beta_5 \texttt{gender}_{i} + \beta_6 \texttt{ethnicity}_{i} + \nu_{it}.$ 



## (a) Getting started

### (i)

Start by running the following code.

In [ ]:
# Load the M348 package
library(M348)
# Load the psid data frame
data(psid)
# Check the data frame has loaded OK
head(psid, 14)
tail(psid, 14)

## (b) Using a pooled estimator

The pooled estimator can be implemented using the `lm()` that you are already familiar with. This is because unless we would refine this estimator to account for the fact that this dataset has a 2-index (or panel) data structure and that observations from the same individual are not independent of each other, pooled OLS returns the same estimates as OLS. Any refinement would mostly change the standard errors and hypothesis testing results.

Recall from Box 23 in Subsection 5.1, when data are pooled the panel structure is ignored in the sense that there are no individual effects in the error term. This means that when specifying the wage equation econometric model, information related to the panel structure is left out.

### (i)

How is the panel structure represented in an econometric model? Which variables in the dataset represent the panel structure?

#### Solution <a class="tocSkip">

The panel structure is represented by the double subscript associated to each variable. $i$ stands for the cross sectional dimension – `id` in the dataset – and $t$ stands for the time series dimension – `period` in the data set.

You will notice that when we specified the wage econometric model, some of the variables only had one index, index $i$. That is because for this dataset, those variables (gender, ethnicity, and educ) do not change for any of the individuals over the course of the data. We call them time invariant and the only variation in them comes from the cross sectional dimension $i$.

By only using one index for all variables, and choosing the index $i$, we are treating all variables, and all observations, as belonging to the same period, just like a cross sectional dataset.

###  <a class="tocSkip">

For a model of the form

$ Y_{it} = \beta_0 + \beta_1 X_{1it} + \cdots + \beta_K X_{Kit} + u_{it}$

an R command for fitting the pooled OLS estimator using `lm()` and saving the results in an object called `pooledModel` is as follows:

`pooledModel <- lm(Y ~ x1 + x2 + ... + xK, data = dataFrame)`.

(Assuming the vectors `Y` and `x1`, `x2`, ..., `xK` are in a data frame called `dataFrame`.)

### (ii)

In the code cell below, remove the `#`, adapt the code to fit the Mincer equation given at the start of this notebook, save the results in an object called `mincerPool` and obtain a summary of the output from fitting the model. (To help you, the code to calculate a new variable giving the values of the square of `exper` also included.)

In [2]:
# psid$experSq <- psid$exper^2
# pooledModel <- lm(Y ~ x1 + x2 + ... + xK, data = dataframe)
# summary(pooledModel)

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

In [3]:
psid$experSq <- psid$exper^2
mincerPool <- lm(wageLog ~ educ+ exper + experSq + occupation + gender + 
                 ethnicity, data = psid)
summary(mincerPool)


Call:
lm(formula = wageLog ~ educ + exper + experSq + occupation + 
    gender + ethnicity, data = psid)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.17807 -0.23858  0.00376  0.24640  2.11848 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.552e+00  4.670e-02 118.884  < 2e-16 ***
educ         5.959e-02  2.645e-03  22.527  < 2e-16 ***
exper        4.199e-02  2.224e-03  18.884  < 2e-16 ***
experSq     -6.896e-04  4.899e-05 -14.078  < 2e-16 ***
occupation  -1.204e-01  1.447e-02  -8.323  < 2e-16 ***
gender      -4.223e-01  1.842e-02 -22.926  < 2e-16 ***
ethnicity   -1.485e-01  2.240e-02  -6.630 3.78e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3628 on 4158 degrees of freedom
Multiple R-squared:  0.383,	Adjusted R-squared:  0.3821 
F-statistic: 430.2 on 6 and 4158 DF,  p-value: < 2.2e-16


### (iii)

Using the results you obtained in part (ii), interpret the effect of education on wages.

#### Solution <a class="tocSkip">

Using all of the data, and a pooled estimator, the effect of education is at 5.96%  and is statistically significant.

## (c) Using the plm package to obtain the pooled estimator

In part (b) you used the `lm()` to obtain the pooled estimator. In this part, we will use a function, `plm()`, in the plm package to obtain this estimator. So the first step is to load the package.

### (i)

Load the plm package by running the following code. (Note that this will not produce any output.)

In [4]:
library(plm)

Before the plm package is used to estimate the pooled estimator, we need to create a panel data frame that contains all the variables in the model we want to estimate.

### (ii)

Use the code cell below to create a panel data frame called `psidPanel` from `psid`. (Look back at Notebook activity A1.5 if you need a reminder of how to create a panel data frame.)

#### Solution <a class="tocSkip">

The required code is given below. Note that running this code will not produce any output.

In [5]:
psidPanel <- pdata.frame(psid, index=c("id", "period"))


###  <a class="tocSkip">
To estimate a pooled OLS estimator using `plm()`, the basic command is:

`panelPooledReg <- plm(Y ~ x1 + x2 + ... +  xK, data = panelDataFrame, effect = "individual", model = "pooling")`.

The argument `Y ~ x1 + x2 + ... + xK` is the specification of the model we wish to estimate using the pooled estimator. 

The argument `data = panelDataFrame` specifies which panel data frame should be used. 

The argument `model = "pooling"` ensures that the pooled OLS estimator is used. 

As you might expect by now, a summary of the output from fitting this model can be obtained by running the command `summary(panelPooledReg)`.

### (iii)

In the code cell below, remove the `#`, adapt the code to fit the Mincer equation given at the start of this notebook, save the results in an object called `mincerPoolUsingPlm` and obtain a summary of the output from fitting the model. 

In [6]:
# panelPooledReg <- plm(Y~ x1 + x2 + ... +  xK, 
#    data=panelDataFrame, effect = "individual", model = "pooling")
# summary(panelPooledReg)

#### Solution <a class="tocSkip">

The required code and resulting output are as follows.

In [7]:
mincerPoolUsingPlm <- plm(wageLog ~ educ + exper + experSq + occupation + 
                    gender + ethnicity, data=psidPanel,  model="pooling")
summary(mincerPoolUsingPlm)

Pooling Model

Call:
plm(formula = wageLog ~ educ + exper + experSq + occupation + 
    gender + ethnicity, data = psidPanel, model = "pooling")

Balanced Panel: n = 595, T = 7, N = 4165

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-2.178067 -0.238579  0.003756  0.246401  2.118482 

Coefficients:
               Estimate  Std. Error  t-value  Pr(>|t|)    
(Intercept)  5.5518e+00  4.6699e-02 118.8841 < 2.2e-16 ***
educ         5.9586e-02  2.6451e-03  22.5272 < 2.2e-16 ***
exper        4.1994e-02  2.2238e-03  18.8843 < 2.2e-16 ***
experSq     -6.8963e-04  4.8986e-05 -14.0780 < 2.2e-16 ***
occupation  -1.2043e-01  1.4470e-02  -8.3231 < 2.2e-16 ***
gender      -4.2230e-01  1.8420e-02 -22.9259 < 2.2e-16 ***
ethnicity   -1.4850e-01  2.2397e-02  -6.6303  3.78e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    886.9
Residual Sum of Squares: 547.2
R-Squared:      0.38302
Adj. R-Squared: 0.38213
F-statistic: 430.213 on 6 and 4158

### (iv)

Compare the results for the pooled estimator that you obtained in parts (b)(ii) and (c)(iii). What do you notice?

#### Solution <a class="tocSkip">

The `plm()` function produced exactly the same output as we obtained using `lm()` model, which is what we would expect as they are both using the same estimator.

## (d) Which function to choose `lm()` or `plm()`?

In parts (b) and (c) you have seen two different functions for estimating a model using a pooled estimator: `lm()` and `plm()`. As you have seen they both achieve the same thing, so to an extent is does not matter which one you choose to use. 

Using the `lm()` function emphasises how the pooled estimator fits with the types of model you learned about in Unit 1 to 5 and is arguably easier to use. However, as you will see in Notebook activities A1.8 and A1.9 the `plm()` function makes it easier to try different types of estimator for panel data (some of which, including the random effects estimator, cannot be estimated using `lm()`). So in practice, econometricians tend to prefer using the `plm()` function rather than `lm()` function.